<a href="https://colab.research.google.com/github/fsenchyna/CSC869_Term_Project/blob/main/beeimage-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# notebook for bee image classification with a convolutional neural network (CNN)